# Chapter 10: Smart Contract Development

---

## 10.1 What are Smart Contracts?

### 10.1.1 Definition and Characteristics

A **smart contract** is a self-executing program that runs on a blockchain when predetermined conditions are met. They are digital agreements written in code that automatically enforce the terms of a contract without requiring intermediaries.

Think of a smart contract as a **vending machine**: you insert the correct amount (condition), select a product, and the machine automatically dispenses it (execution). There's no need for a human cashier—the machine enforces the rules.

```
Traditional Contract vs. Smart Contract:

Traditional Paper Contract:
┌─────────────────────────────────────┐
│                                     │
│  "Party A agrees to pay Party B     │
│   $100 on January 1st."             │
│                                     │
│  ✗ Requires manual enforcement      │
│  ✗ Needs lawyers/courts if breached │
│  ✗ Subject to interpretation        │
│  ✗ Can be altered or destroyed      │
└─────────────────────────────────────┘

Smart Contract (Code):
┌─────────────────────────────────────┐
│  contract Payment {                 │
│    function pay() public {          │
│      require(block.timestamp >=      │
│              deadline);              │
│      recipient.transfer(100 ether); │
│    }                                 │
│  }                                   │
│                                     │
│  ✓ Automatically executed           │
│  ✓ Cannot be altered                │
│  ✓ Transparent rules                │
│  ✓ No intermediaries needed         │
└─────────────────────────────────────┘
```

**Key Characteristics:**

| Characteristic | Description |
|----------------|-------------|
| **Self-Executing** | Code runs automatically when conditions are met |
| **Deterministic** | Given the same input, always produces the same output |
| **Immutable** | Once deployed, the code cannot be changed (with some exceptions) |
| **Transparent** | Code is visible on the blockchain for anyone to audit |
| **Decentralized** | Executes on multiple nodes, no single point of failure |
| **Trustless** | No need to trust a counterparty—trust the code |

### 10.1.2 Self-Executing Code

Smart contracts are **"if-this-then-that"** on the blockchain. They contain functions that are triggered by transactions. When a transaction calls a function, every node in the network executes the same code and reaches the same result, which is then recorded on the blockchain.

```solidity
// A simple self-executing escrow
contract Escrow {
    address public buyer;
    address public seller;
    address public arbiter;
    bool public goodsReceived;

    constructor(address _seller, address _arbiter) payable {
        require(msg.value > 0, "Must send payment");
        buyer = msg.sender;
        seller = _seller;
        arbiter = _arbiter;
    }

    // Called by buyer when goods are received
    function confirmReceipt() public {
        require(msg.sender == buyer, "Only buyer");
        goodsReceived = true;
    }

    // Called by arbiter to release funds
    function release() public {
        require(msg.sender == arbiter, "Only arbiter");
        require(goodsReceived, "Goods not received");
        payable(seller).transfer(address(this).balance);
    }

    // Called by arbiter to refund buyer
    function refund() public {
        require(msg.sender == arbiter, "Only arbiter");
        payable(buyer).transfer(address(this).balance);
    }
}
```

In this example:
- The contract holds funds until conditions are met.
- The code itself enforces the rules—no one can take the money without proper authorization.

### 10.1.3 Trustless Agreements

Smart contracts enable **trustless interactions**—you don't need to trust the other party, only the code. This is revolutionary because it removes the need for intermediaries (banks, lawyers, notaries) and their associated costs, delays, and potential corruption.

```
Comparison of Trust Models:

Traditional Trust:
┌─────────┐     ┌─────────┐     ┌─────────┐
│ Alice   │────▶│  Bank   │────▶│ Bob     │
└─────────┘     └─────────┘     └─────────┘
                    │
              Alice must trust:
              • Bank won't steal funds
              • Bank won't freeze account
              • Bank won't go bankrupt
              • Bank processes correctly

Trustless (Smart Contract):
┌─────────┐                    ┌─────────┐
│ Alice   │───[Blockchain]────▶│ Bob     │
└─────────┘                    └─────────┘
                    │
              Alice trusts:
              • The code (which is public)
              • The blockchain consensus
              • Cryptography
              • No counterparty risk
```

**Benefits of Trustless Agreements:**
- No counterparty risk (they can't break the agreement)
- Lower costs (no intermediaries)
- Faster settlement (minutes vs. days)
- Global accessibility
- Censorship resistance

---

## 10.2 Smart Contract Lifecycle

Understanding the lifecycle of a smart contract is crucial for development. It progresses through several stages from idea to production.

```
Smart Contract Lifecycle:

┌────────────┐    ┌────────────┐    ┌────────────┐    ┌────────────┐
│ 1. Design  │───▶│ 2. Develop │───▶│ 3. Compile │───▶│ 4. Deploy  │
│ & Planning │    │            │    │            │    │            │
└────────────┘    └────────────┘    └────────────┘    └─────┬──────┘
                                                             │
                                                             ▼
┌────────────┐    ┌────────────┐    ┌────────────┐    ┌────────────┐
│ 7. Upgrade │◀───│ 6. Monitor │◀───│ 5. Interact│◀───│    Test    │
│ (if needed)│    │ & Maintain │    │            │    │            │
└────────────┘    └────────────┘    └────────────┘    └────────────┘
```

### 10.2.1 Development

The development phase involves writing the Solidity code, defining the contract's logic, data structures, and events.

```solidity
// Example: Planning a simple wallet contract
contract SimpleWallet {
    // State variables
    address public owner;
    uint256 public withdrawalLimit;

    // Events for logging
    event Deposit(address indexed from, uint256 amount);
    event Withdrawal(address indexed to, uint256 amount);

    // Modifiers for access control
    modifier onlyOwner() {
        require(msg.sender == owner, "Not owner");
        _;
    }

    // Constructor to initialize
    constructor(uint256 _limit) {
        owner = msg.sender;
        withdrawalLimit = _limit;
    }

    // Functions
    function withdraw(uint256 amount) public onlyOwner {
        require(amount <= withdrawalLimit, "Exceeds limit");
        require(amount <= address(this).balance, "Insufficient balance");
        
        payable(owner).transfer(amount);
        emit Withdrawal(owner, amount);
    }

    receive() external payable {
        emit Deposit(msg.sender, msg.value);
    }
}
```

### 10.2.2 Compilation

Solidity code is compiled into **EVM bytecode** and an **ABI (Application Binary Interface)**.

- **Bytecode**: The machine-readable code that runs on the Ethereum Virtual Machine.
- **ABI**: A JSON description of the contract's functions, parameters, and events, used by applications to interact with the contract.

```
Compilation Process:

┌──────────────┐     ┌──────────────┐     ┌──────────────┐
│ Solidity     │────▶│   Compiler   │────▶│   Bytecode   │
│ Contract.sol │     │   (solc)     │     │   (0x...)    │
└──────────────┘     └──────────────┘     └──────────────┘
                           │
                           ▼
                    ┌──────────────┐
                    │     ABI      │
                    │   (JSON)     │
                    └──────────────┘
```

Using Hardhat to compile:
```bash
npx hardhat compile
```

### 10.2.3 Deployment

Deployment is the process of sending a transaction to the blockchain that creates the contract instance. This transaction includes the compiled bytecode and any constructor arguments.

```
Deployment Transaction:

┌─────────────────────────────────────────────────────────────┐
│ Transaction                                                 │
│ {                                                           │
│   from: 0xYourAddress,                                      │
│   to: (empty - contract creation),                          │
│   data: 0x... (bytecode + constructor arguments),           │
│   value: 0 (or some ETH if payable constructor),            │
│   gas: 1000000                                              │
│ }                                                           │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
┌─────────────────────────────────────────────────────────────┐
│ Blockchain                                                  │
│ • Validates transaction                                     │
│ • Executes constructor                                      │
│ • Creates new address for contract                          │
│ • Stores bytecode at that address                           │
└─────────────────────────────────────────────────────────────┘
```

After deployment, the contract has its own address and exists permanently on the blockchain.

### 10.2.4 Interaction

Once deployed, users and other contracts can interact with the contract by sending transactions to its functions.

```
Interaction Types:

┌─────────────────────────────────────────────────────────────┐
│                         CALL                                │
│  (Read-only, no state change, free)                        │
│                                                             │
│  DApp ──(eth_call)──▶ Contract                              │
│                                                             │
│  • balanceOf()                                              │
│  • owner()                                                  │
│  • getNumber()                                              │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│                      TRANSACTION                            │
│  (Modifies state, costs gas, mined)                        │
│                                                             │
│  DApp ──(sendTransaction)──▶ Contract                       │
│                                                             │
│  • transfer()                                               │
│  • mint()                                                   │
│  • update()                                                 │
└─────────────────────────────────────────────────────────────┘
```

### 10.2.5 Upgradability

Smart contracts are immutable by default, but various patterns allow for upgradeability. This is a double-edged sword: it fixes bugs but introduces complexity and trust assumptions.

**Common Upgrade Patterns:**

```
Proxy Pattern:
┌──────────────┐      ┌──────────────┐
│    Proxy     │─────▶│ Implementation│
│   (Storage)  │      │    (Logic)    │
└──────────────┘      └──────────────┘
        │                     │
        │      delegatecall    │
        └──────────────────────┘

• Proxy contract stores data and user's ETH
• Implementation contract contains logic
• Proxy forwards calls via delegatecall
• Owner can point to new implementation
```

```solidity
// Simplified Proxy Pattern
contract Proxy {
    address public implementation;
    address public owner;
    
    constructor(address _impl) {
        owner = msg.sender;
        implementation = _impl;
    }
    
    function upgrade(address _newImpl) public {
        require(msg.sender == owner, "Not owner");
        implementation = _newImpl;
    }
    
    fallback() external payable {
        address impl = implementation;
        require(impl != address(0));
        
        assembly {
            let ptr := mload(0x40)
            calldatacopy(ptr, 0, calldatasize())
            let result := delegatecall(gas(), impl, ptr, calldatasize(), 0, 0)
            let size := returndatasize()
            returndatacopy(ptr, 0, size)
            
            switch result
            case 0 { revert(ptr, size) }
            default { return(ptr, size) }
        }
    }
}
```

**Upgradability Considerations:**
- **Pros**: Can fix bugs, add features, respond to changing requirements.
- **Cons**: Centralization risk (owner can change rules), increased complexity, storage layout must be preserved.

---

## 10.3 Your First Smart Contract

Let's write, deploy, and interact with a simple "Hello World" contract. This will familiarize you with the basic workflow.

### 10.3.1 Hello World Contract

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

/**
 * @title HelloWorld
 * @dev A simple contract that stores and returns a greeting.
 */
contract HelloWorld {
    // State variable to store the greeting
    string private greeting;

    // Event emitted when greeting is updated
    event GreetingChanged(string oldGreeting, string newGreeting, address indexed changer);

    /**
     * @dev Constructor sets the initial greeting
     * @param _greeting The initial greeting string
     */
    constructor(string memory _greeting) {
        greeting = _greeting;
    }

    /**
     * @dev Returns the current greeting
     * @return The greeting string
     */
    function greet() public view returns (string memory) {
        return greeting;
    }

    /**
     * @dev Updates the greeting
     * @param _newGreeting The new greeting string
     */
    function setGreeting(string memory _newGreeting) public {
        string memory oldGreeting = greeting;
        greeting = _newGreeting;
        emit GreetingChanged(oldGreeting, _newGreeting, msg.sender);
    }
}
```

**Line-by-Line Explanation:**

| Line | Description |
|------|-------------|
| `pragma solidity ^0.8.19;` | Specifies compiler version compatibility. |
| `contract HelloWorld {` | Defines the contract. |
| `string private greeting;` | A private state variable—only visible inside contract. |
| `event GreetingChanged(...);` | Declares an event for logging changes. |
| `constructor(string memory _greeting)` | Runs once when contract is deployed, sets initial greeting. |
| `function greet() public view returns (string memory)` | View function returns current greeting (no gas cost when called externally). |
| `function setGreeting(...) public` | Updates greeting and emits event. |

### 10.3.2 Deploying to Remix VM

Follow these steps to deploy in Remix IDE:

1. **Open Remix**: [remix.ethereum.org](https://remix.ethereum.org)
2. **Create a new file**: In the file explorer, click the "Create New File" icon, name it `HelloWorld.sol`.
3. **Paste the code** above into the editor.
4. **Compile**:
   - Click the Solidity compiler tab (third icon).
   - Select compiler version 0.8.19 (or later).
   - Click "Compile HelloWorld.sol".
5. **Deploy**:
   - Click the Deploy & Run tab (fourth icon).
   - Environment: Choose "Remix VM (Cancun)" – a local sandbox blockchain.
   - In the constructor parameters field, enter a greeting like `"Hello, Blockchain!"` (with quotes).
   - Click "Deploy".
6. **Contract appears** under "Deployed Contracts". Expand it to see functions.

```
Deployment in Remix:
┌─────────────────────────────────────────────┐
│ DEPLOY & RUN TRANSACTIONS                   │
│                                             │
│ Environment: Remix VM (Cancun)              │
│ Account: 0x5B3... (100 ETH)                 │
│ Gas Limit: 3000000                          │
│                                             │
│ Deploy                                      │
│ ┌─────────────────────────────────────────┐ │
│ │ HelloWorld - _greeting "Hello, World!" │ │
│ └─────────────────────────────────────────┘ │
│                    │                        │
│                    ▼                        │
│ Deployed Contracts                          │
│ ┌─────────────────────────────────────────┐ │
│ │ HELLOWORLD AT 0XD7... (MEMORY)          │ │
│ │                                         │ │
│ │ greet                                    │ │
│ │ setGreeting                              │ │
│ └─────────────────────────────────────────┘ │
└─────────────────────────────────────────────┘
```

### 10.3.3 Interacting with the Contract

After deployment, you can interact:

- Click `greet` – it will display the initial greeting you set.
- Enter a new greeting in the `setGreeting` field (e.g., `"Hello, Remix!"` with quotes) and click `setGreeting`. This sends a transaction.
- A transaction popup will appear; confirm it.
- After mining (instant in Remix VM), click `greet` again – it returns the updated greeting.

You can also view the emitted event in the Remix console (bottom panel) under "logs".

```
Transaction Receipt:
┌─────────────────────────────────────────────┐
│ Transaction mined and execution succeed    │
│                                            │
│ Transaction Hash: 0xabc...                 │
│ From: 0x5B3...                              │
│ To: HelloWorld.(address)                    │
│ Gas used: 44214                             │
│                                            │
│ Logs:                                       │
│   - GreetingChanged:                        │
│     oldGreeting: "Hello, Blockchain!"       │
│     newGreeting: "Hello, Remix!"            │
│     changer: 0x5B3...                       │
└─────────────────────────────────────────────┘
```

Congratulations! You've just deployed and interacted with your first smart contract.

---

## 10.4 Building a Simple Storage Contract

Now let's build something more practical: a contract that allows users to store and retrieve a value, but with additional features like ownership and access control.

### 10.4.1 Design and Requirements

**Requirements:**
- Only the owner can set the stored value.
- Anyone can read the value.
- Owner can transfer ownership.
- Emit events when value changes or ownership transfers.

**Design:**
```
┌─────────────────────────────────────────────┐
│             SimpleStorage                    │
├─────────────────────────────────────────────┤
│ owner: address                               │
│ storedData: uint256                          │
├─────────────────────────────────────────────┤
│ constructor()                                │
│ onlyOwner modifier                           │
│ setData(uint256) onlyOwner                   │
│ getData() view returns(uint256)               │
│ transferOwnership(address) onlyOwner          │
├─────────────────────────────────────────────┤
│ event DataChanged(address indexed, uint256)  │
│ event OwnershipTransferred(address indexed)  │
└─────────────────────────────────────────────┘
```

### 10.4.2 Implementation Walkthrough

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract SimpleStorage {
    // --- State Variables ---
    address public owner;
    uint256 private storedData;

    // --- Events ---
    event DataChanged(address indexed changer, uint256 newValue);
    event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);

    // --- Modifiers ---
    modifier onlyOwner() {
        require(msg.sender == owner, "Only owner can call this");
        _;
    }

    // --- Constructor ---
    constructor() {
        owner = msg.sender;
        emit OwnershipTransferred(address(0), owner);
    }

    // --- Functions ---
    /**
     * @dev Set the stored value. Only owner.
     * @param x The new value to store
     */
    function setData(uint256 x) public onlyOwner {
        storedData = x;
        emit DataChanged(msg.sender, x);
    }

    /**
     * @dev Get the stored value. Anyone can call.
     * @return The current stored value
     */
    function getData() public view returns (uint256) {
        return storedData;
    }

    /**
     * @dev Transfer ownership to new address. Only owner.
     * @param newOwner The address of the new owner
     */
    function transferOwnership(address newOwner) public onlyOwner {
        require(newOwner != address(0), "New owner cannot be zero address");
        address oldOwner = owner;
        owner = newOwner;
        emit OwnershipTransferred(oldOwner, newOwner);
    }
}
```

**Explanation of Key Parts:**

| Component | Purpose |
|-----------|---------|
| `owner` | Stores the address of the contract owner. |
| `storedData` | The actual data we want to store (private, but readable via getter). |
| `onlyOwner` modifier | Reusable check that restricts access to owner. |
| `setData` | Owner-only function to update the value and emit event. |
| `getData` | Public view function to read the value. |
| `transferOwnership` | Allows owner to transfer control to another address. |
| Events | Provide off-chain applications with notifications of state changes. |

### 10.4.3 Testing in Remix

1. **Deploy** the contract in Remix VM.
2. **Check owner**: Click `owner` – it should show your account.
3. **Get initial data**: Click `getData` – returns 0.
4. **Try setting data** from owner account:
   - Enter a number (e.g., `42`) in `setData` and click.
   - Confirm transaction.
   - `getData` now returns 42.
5. **Test non-owner access**:
   - Switch to a different account in Remix (dropdown at top).
   - Try to call `setData` – it will revert with "Only owner can call this".
   - `getData` still works (public).
6. **Transfer ownership**:
   - Switch back to owner.
   - Call `transferOwnership` with another account address.
   - Now the new owner can call `setData`.
7. **Check events** in the Remix console.

This simple contract demonstrates core concepts: state variables, modifiers, events, and access control.

---

## 10.5 Building a Token Contract

Tokens are the most common smart contract use case. We'll implement an ERC-20 token, the standard for fungible tokens.

### 10.5.1 ERC-20 Standard Overview

ERC-20 defines a standard interface for fungible tokens. It ensures compatibility between different tokens and applications (wallets, exchanges).

**Required Functions:**

```solidity
interface IERC20 {
    function totalSupply() external view returns (uint256);
    function balanceOf(address account) external view returns (uint256);
    function transfer(address to, uint256 amount) external returns (bool);
    function allowance(address owner, address spender) external view returns (uint256);
    function approve(address spender, uint256 amount) external returns (bool);
    function transferFrom(address from, address to, uint256 amount) external returns (bool);

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
}
```

**Optional but common:**
- `name()` – token name (e.g., "MyToken")
- `symbol()` – token symbol (e.g., "MTK")
- `decimals()` – number of decimals (usually 18)

### 10.5.2 Implementing ERC-20 Functions

Let's build a basic ERC-20 token step by step.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract MyToken {
    // --- State Variables ---
    string public name = "MyToken";
    string public symbol = "MTK";
    uint8 public decimals = 18;
    uint256 public totalSupply;

    mapping(address => uint256) public balanceOf;
    mapping(address => mapping(address => uint256)) public allowance;

    // --- Events ---
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);

    // --- Constructor ---
    constructor(uint256 _initialSupply) {
        totalSupply = _initialSupply * 10 ** decimals;
        balanceOf[msg.sender] = totalSupply;
        emit Transfer(address(0), msg.sender, totalSupply);
    }

    // --- Core Functions ---

    function transfer(address to, uint256 amount) external returns (bool) {
        require(to != address(0), "Transfer to zero address");
        require(balanceOf[msg.sender] >= amount, "Insufficient balance");

        balanceOf[msg.sender] -= amount;
        balanceOf[to] += amount;
        emit Transfer(msg.sender, to, amount);
        return true;
    }

    function approve(address spender, uint256 amount) external returns (bool) {
        require(spender != address(0), "Approve to zero address");
        
        allowance[msg.sender][spender] = amount;
        emit Approval(msg.sender, spender, amount);
        return true;
    }

    function transferFrom(address from, address to, uint256 amount) external returns (bool) {
        require(from != address(0), "Transfer from zero");
        require(to != address(0), "Transfer to zero");
        require(balanceOf[from] >= amount, "Insufficient balance");
        require(allowance[from][msg.sender] >= amount, "Insufficient allowance");

        allowance[from][msg.sender] -= amount;
        balanceOf[from] -= amount;
        balanceOf[to] += amount;
        emit Transfer(from, to, amount);
        return true;
    }
}
```

**Explanation:**

| Function | Description |
|----------|-------------|
| `transfer` | Moves `amount` tokens from caller to `to`. Updates balances and emits event. |
| `approve` | Sets allowance for `spender` to spend up to `amount` from caller's balance. |
| `transferFrom` | Moves tokens from `from` to `to` using the allowance mechanism (like an exchange taking payment). |
| `balanceOf` | Returns token balance of an account. |
| `allowance` | Returns the remaining number of tokens that `spender` can spend on behalf of `owner`. |

### 10.5.3 Minting and Burning Tokens

Often, tokens need to be created (minted) or destroyed (burned). These are not part of the core ERC-20 interface but are common extensions.

```solidity
// Add to MyToken contract

// Mint new tokens (only owner in this example)
function mint(address to, uint256 amount) public onlyOwner {
    require(to != address(0), "Mint to zero address");
    
    totalSupply += amount;
    balanceOf[to] += amount;
    emit Transfer(address(0), to, amount);
}

// Burn tokens (anyone can burn their own)
function burn(uint256 amount) public {
    require(balanceOf[msg.sender] >= amount, "Insufficient balance to burn");
    
    balanceOf[msg.sender] -= amount;
    totalSupply -= amount;
    emit Transfer(msg.sender, address(0), amount);
}
```

### 10.5.4 Complete ERC-20 Implementation

Here's a more complete version with ownership, minting, and burning, using OpenZeppelin-like patterns:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract CompleteERC20 {
    // --- State Variables ---
    string private _name;
    string private _symbol;
    uint8 private _decimals;
    uint256 private _totalSupply;

    mapping(address => uint256) private _balances;
    mapping(address => mapping(address => uint256)) private _allowances;

    address public owner;
    
    // --- Modifiers ---
    modifier onlyOwner() {
        require(msg.sender == owner, "Only owner");
        _;
    }

    // --- Events ---
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
    event OwnershipTransferred(address indexed previousOwner, address indexed newOwner);

    // --- Constructor ---
    constructor(string memory name_, string memory symbol_, uint8 decimals_, uint256 initialSupply) {
        _name = name_;
        _symbol = symbol_;
        _decimals = decimals_;
        owner = msg.sender;

        _mint(msg.sender, initialSupply * 10 ** decimals_);
    }

    // --- Public Getters ---
    function name() public view returns (string memory) { return _name; }
    function symbol() public view returns (string memory) { return _symbol; }
    function decimals() public view returns (uint8) { return _decimals; }
    function totalSupply() public view returns (uint256) { return _totalSupply; }
    function balanceOf(address account) public view returns (uint256) { return _balances[account]; }
    function allowance(address owner_, address spender) public view returns (uint256) { 
        return _allowances[owner_][spender]; 
    }

    // --- Core Functions ---
    function transfer(address to, uint256 amount) public returns (bool) {
        _transfer(msg.sender, to, amount);
        return true;
    }

    function approve(address spender, uint256 amount) public returns (bool) {
        _approve(msg.sender, spender, amount);
        return true;
    }

    function transferFrom(address from, address to, uint256 amount) public returns (bool) {
        uint256 currentAllowance = _allowances[from][msg.sender];
        require(currentAllowance >= amount, "ERC20: insufficient allowance");
        
        _transfer(from, to, amount);
        _approve(from, msg.sender, currentAllowance - amount);
        return true;
    }

    // --- Internal Functions ---
    function _transfer(address from, address to, uint256 amount) internal {
        require(from != address(0), "ERC20: transfer from zero");
        require(to != address(0), "ERC20: transfer to zero");
        require(_balances[from] >= amount, "ERC20: insufficient balance");

        _balances[from] -= amount;
        _balances[to] += amount;
        emit Transfer(from, to, amount);
    }

    function _mint(address to, uint256 amount) internal {
        require(to != address(0), "ERC20: mint to zero");
        
        _totalSupply += amount;
        _balances[to] += amount;
        emit Transfer(address(0), to, amount);
    }

    function _burn(address from, uint256 amount) internal {
        require(from != address(0), "ERC20: burn from zero");
        require(_balances[from] >= amount, "ERC20: insufficient balance to burn");

        _balances[from] -= amount;
        _totalSupply -= amount;
        emit Transfer(from, address(0), amount);
    }

    function _approve(address owner_, address spender, uint256 amount) internal {
        require(owner_ != address(0), "ERC20: approve from zero");
        require(spender != address(0), "ERC20: approve to zero");

        _allowances[owner_][spender] = amount;
        emit Approval(owner_, spender, amount);
    }

    // --- Owner Functions ---
    function mint(address to, uint256 amount) public onlyOwner {
        _mint(to, amount);
    }

    function burn(uint256 amount) public {
        _burn(msg.sender, amount);
    }

    function transferOwnership(address newOwner) public onlyOwner {
        require(newOwner != address(0), "New owner cannot be zero");
        address oldOwner = owner;
        owner = newOwner;
        emit OwnershipTransferred(oldOwner, newOwner);
    }
}
```

---

## 10.6 Building an NFT Contract

Non-fungible tokens (NFTs) represent unique assets. The standard is ERC-721.

### 10.6.1 ERC-721 Standard Overview

ERC-721 defines a minimum interface for NFTs. Each token has a unique `tokenId`.

**Required Functions:**

```solidity
interface IERC721 {
    function balanceOf(address owner) external view returns (uint256);
    function ownerOf(uint256 tokenId) external view returns (address);
    function transferFrom(address from, address to, uint256 tokenId) external;
    function safeTransferFrom(address from, address to, uint256 tokenId) external;
    function safeTransferFrom(address from, address to, uint256 tokenId, bytes calldata data) external;
    function approve(address to, uint256 tokenId) external;
    function getApproved(uint256 tokenId) external view returns (address);
    function setApprovalForAll(address operator, bool approved) external;
    function isApprovedForAll(address owner, address operator) external view returns (bool);

    event Transfer(address indexed from, address indexed to, uint256 indexed tokenId);
    event Approval(address indexed owner, address indexed approved, uint256 indexed tokenId);
    event ApprovalForAll(address indexed owner, address indexed operator, bool approved);
}
```

### 10.6.2 NFT Metadata Structure

NFTs usually have metadata (name, description, image, etc.) stored off-chain (e.g., IPFS) with a pointer on-chain.

```solidity
// Typical metadata JSON
{
    "name": "My NFT #1",
    "description": "A unique digital collectible",
    "image": "ipfs://QmZbH...",
    "attributes": [
        {
            "trait_type": "Rarity",
            "value": "Legendary"
        }
    ]
}
```

ERC-721 metadata is accessed via the optional `tokenURI` function:

```solidity
function tokenURI(uint256 tokenId) public view returns (string memory);
```

### 10.6.3 Implementing ERC-721 Functions

Let's build a minimal ERC-721 implementation from scratch to understand the mechanics.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract SimpleNFT {
    // --- State Variables ---
    string public name = "SimpleNFT";
    string public symbol = "SNFT";

    // Token ID to owner
    mapping(uint256 => address) private _owners;
    // Owner to list of owned token IDs (for balanceOf)
    mapping(address => uint256) private _balances;
    // Token ID to approved address
    mapping(uint256 => address) private _tokenApprovals;
    // Owner to operator approvals
    mapping(address => mapping(address => bool)) private _operatorApprovals;

    uint256 private _nextTokenId = 1;

    // --- Events ---
    event Transfer(address indexed from, address indexed to, uint256 indexed tokenId);
    event Approval(address indexed owner, address indexed approved, uint256 indexed tokenId);
    event ApprovalForAll(address indexed owner, address indexed operator, bool approved);

    // --- Core Functions ---
    function balanceOf(address owner) public view returns (uint256) {
        require(owner != address(0), "ERC721: address zero is not a valid owner");
        return _balances[owner];
    }

    function ownerOf(uint256 tokenId) public view returns (address) {
        address owner = _owners[tokenId];
        require(owner != address(0), "ERC721: invalid token ID");
        return owner;
    }

    function approve(address to, uint256 tokenId) public {
        address owner = ownerOf(tokenId);
        require(msg.sender == owner || isApprovedForAll(owner, msg.sender), "Not authorized");

        _tokenApprovals[tokenId] = to;
        emit Approval(owner, to, tokenId);
    }

    function getApproved(uint256 tokenId) public view returns (address) {
        require(_owners[tokenId] != address(0), "ERC721: invalid token ID");
        return _tokenApprovals[tokenId];
    }

    function setApprovalForAll(address operator, bool approved) public {
        require(operator != address(0), "ERC721: operator zero");
        _operatorApprovals[msg.sender][operator] = approved;
        emit ApprovalForAll(msg.sender, operator, approved);
    }

    function isApprovedForAll(address owner, address operator) public view returns (bool) {
        return _operatorApprovals[owner][operator];
    }

    function transferFrom(address from, address to, uint256 tokenId) public {
        require(_isApprovedOrOwner(msg.sender, tokenId), "Not authorized");
        require(_owners[tokenId] == from, "Incorrect owner");

        _transfer(from, to, tokenId);
    }

    function safeTransferFrom(address from, address to, uint256 tokenId) public {
        safeTransferFrom(from, to, tokenId, "");
    }

    function safeTransferFrom(address from, address to, uint256 tokenId, bytes memory data) public {
        transferFrom(from, to, tokenId);
        require(_checkOnERC721Received(from, to, tokenId, data), "Receiver not implemented");
    }

    // --- Internal Helpers ---
    function _isApprovedOrOwner(address spender, uint256 tokenId) internal view returns (bool) {
        address owner = ownerOf(tokenId);
        return (spender == owner || 
                getApproved(tokenId) == spender || 
                isApprovedForAll(owner, spender));
    }

    function _transfer(address from, address to, uint256 tokenId) internal {
        require(to != address(0), "ERC721: transfer to zero");

        // Clear approvals
        delete _tokenApprovals[tokenId];

        // Update balances
        _balances[from] -= 1;
        _balances[to] += 1;
        _owners[tokenId] = to;

        emit Transfer(from, to, tokenId);
    }

    function _mint(address to, uint256 tokenId) internal {
        require(to != address(0), "ERC721: mint to zero");
        require(_owners[tokenId] == address(0), "ERC721: token already minted");

        _balances[to] += 1;
        _owners[tokenId] = to;

        emit Transfer(address(0), to, tokenId);
    }

    function _checkOnERC721Received(address from, address to, uint256 tokenId, bytes memory data) 
        private returns (bool) 
    {
        if (to.code.length > 0) {
            try IERC721Receiver(to).onERC721Received(msg.sender, from, tokenId, data) 
                returns (bytes4 retval) 
            {
                return retval == IERC721Receiver.onERC721Received.selector;
            } catch {
                return false;
            }
        } else {
            return true;
        }
    }

    // --- Minting function ---
    function mint(address to) public returns (uint256) {
        uint256 tokenId = _nextTokenId++;
        _mint(to, tokenId);
        return tokenId;
    }
}

// Interface for contracts that receive NFTs
interface IERC721Receiver {
    function onERC721Received(
        address operator,
        address from,
        uint256 tokenId,
        bytes calldata data
    ) external returns (bytes4);
}
```

### 10.6.4 Complete ERC-721 Implementation

A production-ready NFT contract would include metadata support and possibly royalties. Here's a more complete version with tokenURI:

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

contract MyNFT {
    // Inherit from a base implementation (or use OpenZeppelin)
    // For brevity, we'll extend the previous SimpleNFT with metadata

    string private _baseURI;
    mapping(uint256 => string) private _tokenURIs;

    constructor(string memory name_, string memory symbol_, string memory baseURI_) {
        name = name_;
        symbol = symbol_;
        _baseURI = baseURI_;
    }

    function _setTokenURI(uint256 tokenId, string memory _tokenURI) internal {
        require(_owners[tokenId] != address(0), "ERC721: URI set of nonexistent token");
        _tokenURIs[tokenId] = _tokenURI;
    }

    function tokenURI(uint256 tokenId) public view returns (string memory) {
        require(_owners[tokenId] != address(0), "ERC721: URI query for nonexistent token");
        
        string memory _tokenURI = _tokenURIs[tokenId];
        string memory base = _baseURI;

        // If base is empty, just return tokenURI
        if (bytes(base).length == 0) {
            return _tokenURI;
        }
        // If tokenURI is empty, return base + tokenId
        if (bytes(_tokenURI).length == 0) {
            return string(abi.encodePacked(base, Strings.toString(tokenId)));
        }
        // If both exist, return concatenation (or whichever)
        return _tokenURI;
    }

    function mintWithURI(address to, string memory uri) public returns (uint256) {
        uint256 tokenId = _nextTokenId++;
        _mint(to, tokenId);
        _setTokenURI(tokenId, uri);
        return tokenId;
    }
}

// Helper library for converting uint to string
library Strings {
    function toString(uint256 value) internal pure returns (string memory) {
        // ... implementation (omitted for brevity)
    }
}
```

---

## 10.7 Smart Contract Security

Security is paramount in smart contract development because bugs can lead to irreversible loss of funds.

### 10.7.1 Reentrancy Attacks

**The Attack:** A malicious contract re-enters the calling function before the first invocation finishes, exploiting intermediate state.

```solidity
// VULNERABLE CONTRACT
contract Vulnerable {
    mapping(address => uint) public balances;

    function withdraw() public {
        uint bal = balances[msg.sender];
        require(bal > 0);

        (bool success, ) = msg.sender.call{value: bal}(""); // Send ETH
        require(success, "Transfer failed");

        balances[msg.sender] = 0; // Update AFTER external call
    }
}

// ATTACK CONTRACT
contract Attacker {
    Vulnerable vulnerable;
    
    constructor(address _vulnerable) {
        vulnerable = Vulnerable(_vulnerable);
    }

    function attack() public payable {
        vulnerable.withdraw();
    }

    receive() external payable {
        if (address(vulnerable).balance >= 1 ether) {
            vulnerable.withdraw(); // Re-enter before balance is set to 0
        }
    }
}
```

**Prevention:**
- Use **Checks-Effects-Interactions** pattern: make all state changes before external calls.
- Use OpenZeppelin's `ReentrancyGuard` modifier.

```solidity
// Secure version
contract Secure {
    mapping(address => uint) public balances;
    bool internal locked;

    modifier noReentrant() {
        require(!locked, "No reentrancy");
        locked = true;
        _;
        locked = false;
    }

    function withdraw() public noReentrant {
        uint bal = balances[msg.sender];
        require(bal > 0);

        balances[msg.sender] = 0; // CHECK (balance>0) then EFFECTS (update state) then INTERACTION
        (bool success, ) = msg.sender.call{value: bal}("");
        require(success, "Transfer failed");
    }
}
```

### 10.7.2 Integer Overflow/Underflow

Before Solidity 0.8.x, arithmetic could overflow/underflow. Now it's checked by default, but you still need to be aware.

```solidity
// Pre-0.8.x vulnerable code
function add(uint a, uint b) public pure returns (uint) {
    return a + b; // could overflow
}

// After 0.8.x, this reverts on overflow
function addSafe(uint a, uint b) public pure returns (uint) {
    return a + b; // safe
}
```

For older versions, use SafeMath library.

### 10.7.3 Denial of Service (DoS)

Attackers can make a contract unusable.

**Examples:**
- Block gas limit by iterating over an unbounded array.
- Making `receive` function always revert, preventing fund transfers.
- Front-running to prevent certain operations.

**Prevention:**
- Avoid loops over dynamic arrays that can grow large.
- Use pull over push for payments (let users withdraw instead of pushing to them).
- Implement circuit breakers (pause functionality).

### 10.7.4 Front-Running

Attackers see pending transactions in the mempool and submit their own with higher gas to get executed first.

**Examples:**
- Buying a token before a large purchase drives price up (sandwich attack).
- Seeing a profitable trade and copying it with higher gas.

**Mitigation:**
- Use commit-reveal schemes.
- Use submarine sends (private mempools).
- Set slippage tolerance.

### 10.7.5 Access Control Vulnerabilities

Missing or incorrect access control can allow anyone to call sensitive functions.

```solidity
// VULNERABLE
function mint(address to, uint amount) public {
    _mint(to, amount); // anyone can mint!
}

// SECURE
function mint(address to, uint amount) public onlyOwner {
    _mint(to, amount);
}
```

**Prevention:** Always use modifiers like `onlyOwner`, and carefully design role-based access.

### 10.7.6 Best Practices and Security Patterns

| Pattern | Description |
|---------|-------------|
| **Checks-Effects-Interactions** | Validate inputs, update state, then interact externally. |
| **Pull over Push** | Let users withdraw funds rather than pushing to them. |
| **Circuit Breaker** | Add a pause mechanism to stop functionality in emergencies. |
| **Rate Limiting** | Prevent rapid repeated calls. |
| **Emergency Stop** | Allow owner to pause contract. |
| **Use Established Libraries** | OpenZeppelin provides audited implementations. |

**Checklist for Deployment:**
- [ ] All functions have proper access control.
- [ ] No reentrancy vulnerabilities.
- [ ] Arithmetic uses SafeMath or Solidity 0.8+.
- [ ] No unbounded loops.
- [ ] Events emitted for important state changes.
- [ ] Contract tested thoroughly.
- [ ] Code audited (for high-value contracts).

---

## Chapter Summary

| Section | Key Takeaways |
|---------|---------------|
| **What are Smart Contracts?** | Self-executing code on blockchain, trustless, immutable, transparent. |
| **Lifecycle** | Design → Develop → Compile → Deploy → Interact → Monitor → Upgrade. |
| **First Contract** | HelloWorld demonstrates basic syntax and Remix workflow. |
| **Simple Storage** | Introduces ownership, events, and modifiers. |
| **ERC-20 Token** | Standard fungible token with transfer, approve, transferFrom. |
| **ERC-721 NFT** | Non-fungible token standard, each token unique. |
| **Security** | Reentrancy, overflow, DoS, front-running, access control – use best practices. |

**Next Chapter Preview:** Chapter 11 – Development Tools and Frameworks. We'll explore Hardhat, Foundry, Truffle, and how to streamline development with testing, deployment scripts, and advanced features.